<a href="https://colab.research.google.com/github/fedhere/MLTSA_FBianco/blob/main/MLTSA_sam_freeze_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

In [7]:
cd /content/drive/Shareddrives/FASTlab/lightEchoes/SAMsclub

/content/drive/Shareddrives/FASTlab/lightEchoes/SAMsclub


In [8]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import Resize
from PIL import Image
import torch
import torch.nn.functional as F
import os
import cv2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
import matplotlib.pyplot as plt
%matplotlib inline

### generate masks

In [10]:
sam = sam_model_registry["vit_h"](checkpoint="sam_vit_h_4b8939.pth")

sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:


if type(uploaded) is not dict: uploaded = uploaded.files  ## Deal with filedit versions
file_contents = uploaded[list(uploaded.keys())[0]]
file_contents

In [28]:
# prompt: read in the image uploaded into files content

import matplotlib.pyplot as plt
from PIL import Image
import io

image = Image.open(io.BytesIO(file_contents))

image = np.array(image)


In [ ]:
image.shape

In [ ]:
plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.axis('off')
plt.show()

In [31]:
masks = mask_generator.generate(image)

In [ ]:
masks[2]

In [ ]:
masks[0]['segmentation'].shape

In [ ]:
plt.imshow(masks[0]['segmentation'])

In [35]:
sam.mask_decoder.transformer.layers

ModuleList(
  (0-1): 2 x TwoWayAttentionBlock(
    (self_attn): Attention(
      (q_proj): Linear(in_features=256, out_features=256, bias=True)
      (k_proj): Linear(in_features=256, out_features=256, bias=True)
      (v_proj): Linear(in_features=256, out_features=256, bias=True)
      (out_proj): Linear(in_features=256, out_features=256, bias=True)
    )
    (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (cross_attn_token_to_image): Attention(
      (q_proj): Linear(in_features=256, out_features=128, bias=True)
      (k_proj): Linear(in_features=256, out_features=128, bias=True)
      (v_proj): Linear(in_features=256, out_features=128, bias=True)
      (out_proj): Linear(in_features=128, out_features=256, bias=True)
    )
    (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (mlp): MLPBlock(
      (lin1): Linear(in_features=256, out_features=2048, bias=True)
      (lin2): Linear(in_features=2048, out_features=256, bias=True)
      (act): ReLU()
  

In [36]:
sam

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1280, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-31): 32 x Block(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=1280, out_features=5120, bias=True)
          (lin2): Linear(in_features=5120, out_features=1280, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(1280, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d

In [37]:
from prettytable import PrettyTable
def count_parameters(model):
    table = PrettyTable(['Modules', 'Parameters'])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f'Total Trainable Params: {total_params}')
    return total_params

In [38]:
count_parameters(sam)

+-----------------------------------------------------------------------------+------------+
|                                   Modules                                   | Parameters |
+-----------------------------------------------------------------------------+------------+
|                           image_encoder.pos_embed                           |  5242880   |
|                    image_encoder.patch_embed.proj.weight                    |   983040   |
|                     image_encoder.patch_embed.proj.bias                     |    1280    |
|                     image_encoder.blocks.0.norm1.weight                     |    1280    |
|                      image_encoder.blocks.0.norm1.bias                      |    1280    |
|                    image_encoder.blocks.0.attn.rel_pos_h                    |    2160    |
|                    image_encoder.blocks.0.attn.rel_pos_w                    |    2160    |
|                    image_encoder.blocks.0.attn.qkv.weight           

641090608

In [ ]:
while 1:pass

### load dataset

In [ ]:
# Define dataset with masks
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, mask_dir, transform=None):
        self.root_dir = root_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(self.root_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.images[idx])
        mask_name = os.path.join(self.mask_dir, self.images[idx])

        image = Image.open(img_name)
        mask = Image.open(mask_name)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask


In [ ]:
sam.named_parameters()

In [ ]:
from prettytable import PrettyTable
def count_trainablelayers(model):
    trainable = 0
    table = PrettyTable(['Modules', 'Gradient'])

    for name, parameter in model.named_parameters():
        table.add_row([name, parameter.requires_grad])
        trainable +=1
    print(table)

    return trainable

count_trainablelayers(sam)

In [ ]:
def freeze_layer(model, layernameroot):
    trainable = 0
    table = PrettyTable(['Modules', 'Gradient'])

    for name, parameter in model.named_parameters():
        if not name.startswith(layernameroot):
          #print("here", name)
          parameter.requires_grad = False
        table.add_row([name, parameter.requires_grad])
        if parameter.requires_grad:
          trainable +=1
    print(table)

    return trainable

ntrainable = freeze_layer(sam, 'mask_decoder.iou_prediction_head')
torch.save(model.state_dict(), f"samLE_funfrozen{ntrainable}.pth")